In [ ]:
import math
import numpy as np
import pprint as pp

from common import *
from mcs_shared import load_als_depth, load_isnobal_depth, ALL_FLIGHT_DATES

from bokeh.models.formatters import BasicTickFormatter

In [ ]:
use_hvplot()

In [ ]:
%load_ext autoreload
%autoreload 2

## Snow depth map resolution

In [ ]:
RESOLUTION = 10
BASE_RUN = True

In [ ]:
def diff_violin_mpl(data, date):
    fig  = plt.figure(dpi=300, figsize=(1,2))
    ax = fig.gca()
    
    parts = ax.violinplot(
        data[~np.isnan(data)],
        points=1000,
        quantiles=[0.01, 0.99],
        showextrema=False,
        # showmeans=True,
        showmedians=True,
    )
    for pc in parts['bodies']:
        if BASE_RUN:
            color = "indigo"
        else:
            color = "darkorange"
        pc.set_facecolor(color)
        pc.set_alpha=0.95
        pc.set_edgecolor('black')

    parts['cmedians'].set_color(['black']) 
    parts['cquantiles'].set_color('dimgrey')
    
    ax.set_xticklabels([])
    ax.set_ylim(-2, 2)
    # ax.set_yticks(np.arange(-2, 2.2, step=0.2))
    ax.yaxis.minorticks_on()
    ax.set_ylabel('Depth Difference (m)')
    # ax.set_title(date)

    return fig
    
def diff_area_mpl(data, date):
    x_ticks = np.arange(0, data.shape[1], 100)
    y_ticks = np.arange(0, data.shape[0], 100)
    x_labels = (np.arange(0, data.shape[1] * RESOLUTION, 1000) / 1000).astype(int)
    y_labels = (np.arange(0, data.shape[0] * RESOLUTION, 1000) / 1000).astype(int)

    fig  = plt.figure(dpi=300)
    ax = fig.gca()
    img = ax.imshow(
        np.flipud(data), 
        aspect='equal', origin='lower',
        vmin=-1, vmax=1, cmap='bwr_r',
    )
    ax.set_title('')
    at = AnchoredText(
        pd.to_datetime(date).strftime("%Y-%m-%d"), 
        prop=dict(size=10), 
        frameon=False, 
        loc='upper left', 
        pad=0.3, 
        borderpad=0.25,
    )
    ax.add_artist(at)
    ax.set_xlabel('Easting [km]')
    ax.set_xticks(x_ticks, labels=x_labels)
    ax.set_ylabel('Northing [km]')
    ax.set_yticks(y_ticks, labels=y_labels)

    cbar = fig.colorbar(img, label='Depth Difference (m)', shrink=1.0, pad=0.01, aspect=30)
    cbar.ax.tick_params(labelsize=8)
    
    return fig

Difference = $ {iSnobal} - {ALS} $

In [ ]:
def plot_flight(date):
    mcs_als = load_als_depth(date, RESOLUTION, base_run=BASE_RUN)
    isnobal = load_isnobal_depth(date, RESOLUTION, base_run=BASE_RUN)
    diff = (isnobal - mcs_als)
    diff_flatten = diff.flatten()
    diff_flatten = diff_flatten[~np.isnan(diff_flatten)]

    print("\n\n" + date)
    display(pd.Series(diff_flatten).describe(percentiles=[0.01, 0.5, 0.99]))
    
    violin = diff_violin_mpl(diff_flatten, date)
    area = diff_area_mpl(diff, date)

    return violin.show(), area.show()

def plot_hv():
    x_coords = np.arange(0, mcs_als.shape[1] * RESOLUTION, RESOLUTION)
    y_coords = np.arange(0, mcs_als.shape[0] * RESOLUTION, RESOLUTION)
    
    hist_bins = np.arange(0, 5, 0.1)
    
    summary_stats(mcs_als, isnobal, diff_flatten)
    
    return hv.Layout([
            # mcs_als.snowdepth.hvplot(x='x', y='y', clim=(0,3)).opts(title='ALS') + 
            # isnobal.thickness.hvplot(clim=(0,3)).opts(title='Model') + 
            hv.Image(
                (x_coords, y_coords, diff)
            ).opts(
                title='Depth Difference', **BOKEH_FONT,
                tools=['hover'],
                cmap='bwr_r', clim=(-1,1), colorbar=True,
                height=600, width=800, aspect='equal',
                yformatter=BasicTickFormatter(use_scientific=False), xformatter=BasicTickFormatter(use_scientific=False),
            ),
            hv.Violin(diff_flatten, label='Difference').opts(
                title='', inner='quartiles', 
                ylabel='Snow Depth (m)', xlabel='', ylim=(-1.5, 1.5),
                height=600, width=600, **BOKEH_FONT
            ), 
        ] + [
            # mcs_als.snowdepth.hvplot.density(alpha=.3, label='ALS').opts(title='Depth Distribution') *
            # mcs_als.snowdepth.rename('ALS').hvplot.hist(bins=hist_bins, alpha=.5, height=400, width=800, color='orange', title='Snow Depth') * \
            # isnobal.thickness.rename('iSnobal').hvplot.hist(bins=hist_bins, alpha=.5, height=400, width=800, **BOKEH_FONT)
        ]).cols(2).opts(title=date, **HV_PLOT_OPTS, shared_axes=False)


def summary_stats(mcs_als, isnobal, diff):
    stats = [
        get_stats("ALS", mcs_als),
        get_stats("iSnobal", isnobal),
        get_stats("Difference", diff),
    ]
    display(pd.concat(stats))


def get_stats(source, data):
    return pd.DataFrame({
        "Source": [source], 
        "Median": [np.nanmedian(data).round(2)],
        "Mean": [np.nanmean(data).round(2)],
        "SD": [np.nanstd(data).round(2)],
    })

# Flights

In [ ]:
[plot_flight(flight) for flight in ALL_FLIGHT_DATES ];